<a href="https://colab.research.google.com/github/availe/Spystat-Engine-Colab/blob/main/StatsQuant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Project setup

!pip install itables > /dev/null 2>&1
!pip install feature-engine > /dev/null 2>&1

from pathlib import Path
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)

repo_dir = "/content/Spystat-Engine-Colab"

if Path(repo_dir).exists():
    !git -C Spystat-Engine-Colab pull
else:
    !git clone https://github.com/availe/Spystat-Engine-Colab.git

data_path = Path('/content/Spystat-Engine-Colab/data/')

Cloning into 'Spystat-Engine-Colab'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 103 (delta 39), reused 86 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 2.95 MiB | 14.73 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [23]:
#@title Merge daily datasets and generate financial features

from feature_engine.creation import CyclicalFeatures

# Load and merge the datasets
file_names = ['dxy_nyb_us_dollar_index.csv', 'fvy_t_yield_5_years.csv',
              'irx_13_week_t_bill.csv', 'spy_dividends.csv', 'spy.csv',
              'tnx_interest_rate_10_year.csv', 'unrate.csv', 'vix.csv']


dfs = []

for file in file_names:
  temp_df = pd.read_csv(data_path / file)
  temp_df.columns = temp_df.columns.str.lower()
  prefix = file[:-4].lower()
  temp_df.columns = [col if col == 'date' or col.startswith(prefix) else f'{prefix}_{col}' for col in temp_df.columns]
  dfs.append(temp_df)

df = dfs[0]
for temp_df in dfs[1:]:
    df = pd.merge(df, temp_df, on='date', how='outer')



# Calculate technical indicators
def calculate_metrics(df, instrument, window):
    # Moving Average
    ma_col = f'{instrument}_{window}_ma_close'
    df[ma_col] = df[f'{instrument}_close'].rolling(window=window).mean()
    # Oscillator
    df[f'{instrument}_oscillator_{window}'] = ((df[f'{instrument}_close'] - df[ma_col]) / df[ma_col] * 100)
    # Volume Z-Score
    volume_col = f'{instrument}_volume'
    if volume_col in df.columns:
        df[f'{instrument}_volume_z_{window}'] = (
            (df[volume_col] - df[volume_col].rolling(window=window).mean()) /
            df[volume_col].rolling(window=window).std()
        )

calculate_metrics(df, 'spy', 5)
calculate_metrics(df, 'spy', 20)
calculate_metrics(df, 'vix', 20)

# Calculate percentage changes
df['spy_percent_close_change'] = df['spy_close'].pct_change(fill_method=None) * 100 # check this
df['spy_percent_change:high_to_previous_close'] = (df['spy_high'] - df['spy_close'].shift(1)) / df['spy_close'].shift(1) * 100
df['spy_percent_change:low_to_previous_close'] = (df['spy_low'] - df['spy_close'].shift(1)) / df['spy_close'].shift(1) * 100

# Create labels for prediction
label_cols = [
    'spy_percent_close_change',
    'spy_percent_change:high_to_previous_close',
    'spy_percent_change:low_to_previous_close'
]
df[[f'{col}_label' for col in label_cols]] = df[label_cols].shift(-1)

df.dropna(subset=[f'{col}_label' for col in label_cols], inplace=True)

In [29]:
#@title Merge with monthly datasets and generate temporal features

cpi = pd.read_csv(data_path / 'cpi.csv')
ppi = pd.read_csv(data_path / 'ppi.csv')

show(ppi)

# show(df, lengthMenu=[5, 10, 25, 50, 100], paging=True)

In [ ]:
#@title Dimensionality reduction

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

df.dropna(inplace=True)
show(df, lengthMenu=[5, 10, 25, 50, 100], paging=True)
print(df.shape)

columns_to_drop = [
    'spy_close', 'spy_adj close', 'spy_percent_close_change',
    'spy_percent_change:high_to_previous_close',
    'spy_percent_change:low_to_previous_close',
    'spy_percent_close_change_label',
    'spy_percent_change:high_to_previous_close_label',
    'spy_percent_change:low_to_previous_close_label'
]
X = df.drop(columns=columns_to_drop)
y = df['spy_percent_close_change_label']

pipe = Pipeline([
    ("scale", StandardScaler()),
    ("model", RandomForestRegressor())
])

param_grid = {
    'model__n_estimators': [100],          # Number of trees
    # 'model__max_depth': [None, 10],        # Maximum tree depth
    # 'model__min_samples_split': [2, 5],    # Minimum samples to split
    # 'model__min_samples_leaf': [1, 2],     # Minimum samples per leaf
    # 'model__bootstrap': [True, False]      # Bootstrap samples
}

# Set up GridSearchCV
model = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, error_score='raise')

# Fit the model


(7902, 50)


In [ ]:
new = pd.read_csv(data_path/'cpi.csv')
new2 = pd.read_csv(data_path/'cpi.csv')
display(new)

Year      Jan      Feb      Mar      Apr      May      Jun      Jul  \
0    1913    9.800    9.800    9.800    9.800    9.700    9.800    9.900   
1    1914   10.000    9.900    9.900    9.800    9.900    9.900   10.000   
2    1915   10.100   10.000    9.900   10.000   10.100   10.100   10.100   
3    1916   10.400   10.400   10.500   10.600   10.700   10.800   10.800   
4    1917   11.700   12.000   12.000   12.600   12.800   13.000   12.800   
..    ...      ...      ...      ...      ...      ...      ...      ...   
107  2020  257.971  258.678  258.115  256.389  256.394  257.797  259.101   
108  2021  261.582  263.014  264.877  267.054  269.195  271.696  273.003   
109  2022  281.148  283.716  287.504  289.109  292.296  296.311  296.276   
110  2023  299.170  300.840  301.836  303.363  304.127  305.109  305.691   
111  2024  308.417  310.326  312.332  313.548  314.069  314.175  314.540   

         Aug      Sep      Oct      Nov      Dec    HALF1    HALF2  
0      9.900   10.000   10.000   10.100   10.000      NaN      NaN  
1     10.200   10.200   10.100   10.200   10.100      NaN      NaN  
2     10.100   10.100   10.200   10.300   10.300      NaN      NaN  
3     10.900   11.100   11.300   11.500   11.600      NaN      NaN  
4     13.000   13.300   13.500   13.500   13.700      NaN      NaN  
..       ...      ...      ...      ...      ...      ...      ...  
107  259.918  260.280  260.388  260.229  260.474  257.557  260.065  
108  273.567  274.310  276.589  277.948  278.802  266.236  275.703  
109  296.171  296.808  298.012  297.711  296.797  288.347  296.963  
110  307.026  307.789  307.671  307.051  306.746  302.408  306.996  
111      NaN      NaN      NaN      NaN      NaN  312.145      NaN  

[112 rows x 15 columns]